sedfitter is a Python port of the SED fitting tool described in [Robitaille+ (2007)](). It is also the primary interface for the [Robitaille+ (2006)](https://ui.adsabs.harvard.edu/abs/2006ApJS..167..256R/exportcitation) and [Robitaille (2017)](https://ui.adsabs.harvard.edu/abs/2017A%26A...600A..11R/abstract) sets of radiative transfer YSO models. A [new version](https://zenodo.org/records/10522816) of the latter set was recently released alongside [Richardson+ (2024)](https://ui.adsabs.harvard.edu/abs/2024ApJ...961..188R/abstract) (R24). This version expands the available information, but useof these models will likely require slight modifications to a sedfitter-based workflow in order to maintain functionality.

The purpose of this notebook is to synthesize aspects of the [sedfitter documentation](https://sedfitter.readthedocs.io/en/stable/) into a single demonstration of the workflow and basic functionality of the tool, as well as to provide additional information on how to work with the R24 version of the 2017 YSO models. It is current as of PR #(number).

In [1]:
import os
import glob

import numpy as np
from astropy import units as u

from sedfitter import (fit, plot, plot_params_1d, plot_params_2d,
                       write_parameters, write_parameter_ranges)
from sedfitter.extinction import Extinction

In [2]:
%rm -rf r24_example
%mkdir r24_example

which model directory to point to?

In R24, the models are contained within directories named for several "model geometries", which are subsets of models with common features. Table 2 of R17 contains a full accounting of the names and significances of these geometries. For the purposes of this notebook, we pick "spubhmi", which is the most populous geometry. "spubhmi" models all have disks, rotationally flattened envelopes, and bipolar cavities

In [3]:
# Define path to models
r24_modeldir = '/blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2'
geometry = 'spubhmi'

summarize extinction

In [4]:
# Read in extinction law. We read in columns 1 (the wavelength in microns) and
# 4 (the opacity in cm^2/g)
extinction = Extinction.from_file('kmh94.par', columns=[0, 3],
                                  wav_unit=u.micron, chi_unit=u.cm**2 / u.g)

Both the R17 and R24 sets of models have convolved their SEDs with the response profiles of several current or formerly active instruments often used to observe YSOs. One of the main differences between the 2017 and 2024 releases of the YSO models is in how their filters are named; the 2017 release named them solely by the filter itself, while the 2024 version incorporates [astroquery](https://astroquery.readthedocs.io/en/latest/) into its convolution procedure to retrieve filter profiles from the [SVO FPS](https://svo2.cab.inta-csic.es/theory/fps/). Consequently, when fitting using the 2024 models, the convolved fluxes must be invoked using the name as formatted by the SVO FPS; generally, this naming convention is {instrument}/{camera}.{filter}. In this data, we have access to 2MASS's J/H/K filters and the Spitzer IRAC's I1-I4.

In [5]:
# Define filters
filters = ['2MASS/2MASS.J', '2MASS/2MASS.H', '2MASS/2MASS.Ks', 
           'Spitzer/IRAC.I1', 'Spitzer/IRAC.I2', 
           'Spitzer/IRAC.I3', 'Spitzer/IRAC.I4']

describe the apertures

In [6]:
# Define apertures
apertures = [3., 3., 3., 3., 3., 3., 3.] * u.arcsec

anything extra for fitting? describe the data?

In [7]:
# Run the fitting
fit('data_glimpse', filters, apertures, f'{r24_modeldir}/{geometry}',
    'r24_example/output.fitinfo',
    extinction_law=extinction,
    distance_range=[1., 2.] * u.kpc,
    av_range=[0., 40.])

 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  spubhmi
   Log[d] stepping     :  0.02
   Number of distances :  17

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

Data shape=(720000, 17, 7).  use_memmap=True
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/2MASS/2MASS.J.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/2MASS/2MASS.H.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/2MASS/2MASS.Ks.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/Spitzer/IRAC.I1.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/Spitzer/IRAC.I2.fits


Now that we have performed the fit, we have a set of FitInfo objects that contain the results. Oftentimes when working with these, it is desirable to limit the number of models being considered. sedfitter provides a number of ways to do this: we provide a general overview here.

For this example, we will limit consideration to models which exhibit a $\chi^2$-per-data-point $<$ 3.

In [8]:
select_format = ('F', 3)

plot!

In [9]:
# Make SED plots
plot('r24_example/output.fitinfo', 'r24_example/plots_seds', plot_max=100, select_format=select_format)

plot the parameter histograms!

In [10]:
# Make histograms of the source luminosity
plot_params_1d('r24_example/output.fitinfo', 'Source Luminosity', 
               output_dir='r24_example/plots_lum',
               log_x=True, select_format=select_format)

introduce the array parameters/handling

In [11]:
# Make histograms of the sphere mass in the smallest aperture
plot_params_1d('r24_example/output.fitinfo', 'Sphere Masses',aperture=0,
               output_dir='r24_example/plots_sphmass_smallap',
               log_x=True, select_format=select_format)

describe all the apertures

In [12]:
all_apertures = np.logspace(2,6,20) * u.AU

find the "right" aperture

In [13]:
mid_distance_ap = 4500 * u.AU
ap_num = np.argmin(abs(mid_distance_ap - all_apertures))

replot

In [14]:
# Make histograms of the sphere mass in the smallest aperture
plot_params_1d('r24_example/output.fitinfo', 'Sphere Masses',aperture=ap_num,
               output_dir='r24_example/plots_sphmass_fitap',
               log_x=True, select_format=select_format)

and you can also do this in 2d

In [15]:
# Make 2-d plots of the source luminosity vs circumstellar mass, 
# in an aperture consistent with the size used for fitting
plot_params_2d('r24_example/output.fitinfo', 
               'Source Luminosity', 'Sphere Masses',
               output_dir='r24_example/plots_lum_sphmass_fitap',
               log_x=True, log_y=True, select_format=select_format,
               aperture=ap_num)

you can also write the parameters to a file; array parameters work the same 

In [ ]:
# Write out all models with a delta chi^2-chi_best^2 per datapoint < 3
write_parameters('r24_example/output.fitinfo', 'r24_example/parameters.txt',
                 select_format=select_format,
                 aperture=ap_num)

/blue/adamginsburg/richardson.t/dev/sedfitter/sedfitter/write_parameters.py:102: UserWarning: Warning: converting a masked element to nan.
  fout.write('%10.3e ' % (vals[aperture]))
/blue/adamginsburg/richardson.t/dev/sedfitter/sedfitter/write_parameters.py:104: UserWarning: Warning: converting a masked element to nan.
  fout.write('%10.3e ' % (vals[0]))


finally, you can write out the range of parameters and the best-fit values to a file; array parameters work the same

In [ ]:
# Write out the min/max ranges corresponding to the above file
write_parameter_ranges('r24_example/output.fitinfo', 'r24_example/parameter_ranges.txt',
                       select_format=select_format,
                       aperture=ap_num)